In [370]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from fancyimpute import IterativeImputer
import math
%matplotlib inline

In [371]:
train = pd.read_csv('/Users/owner/Desktop/2019國泰大數據競賽/Cathay_BigData-Competition/Data/train.csv',encoding = 'big5')
test = pd.read_csv('/Users/owner/Desktop/2019國泰大數據競賽/Cathay_BigData-Competition/Data/test.csv',encoding = 'big5')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [372]:
train.describe()

,CUS_ID,EDUCATION_CD,MARRIAGE_CD,L1YR_A_ISSUE_CNT,L1YR_B_ISSUE_CNT,CHANNEL_A_POL_CNT,CHANNEL_B_POL_CNT,OCCUPATION_CLASS_CD,APC_CNT,INSD_CNT,...,OUTPATIENT_SURGERY_AMT,INPATIENT_SURGERY_AMT,PAY_LIMIT_MED_MISC_AMT,FIRST_CANCER_AMT,ILL_ACCELERATION_AMT,ILL_ADDITIONAL_AMT,LONG_TERM_CARE_AMT,MONTHLY_CARE_AMT,LIFE_INSD_CNT,L1YR_GROSS_PRE_AMT
count,1.000000e+05,79438.000000,92049.000000,100000.000000,100000.000000,100000.000000,100000.000000,96040.000000,100000.000000,100000.000000,...,72460.000000,72460.000000,72460.000000,72460.000000,72460.000000,72460.000000,72460.000000,72460.000000,100000.000000,100000.000000
mean,1.267299e+06,2.169075,0.315289,0.118170,0.003330,1.627730,0.031080,1.304967,0.681070,0.283620,...,0.089922,0.058821,0.094473,0.014284,0.004284,0.002229,0.002394,0.018912,0.031473,0.000525
std,7.382534e+05,0.929452,0.489678,0.561755,0.062442,3.365798,0.262326,0.711769,0.614847,0.747238,...,0.101751,0.078278,0.166933,0.033439,0.014163,0.009714,0.020305,0.040821,0.034269,0.003117
min,1.200000e+01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.275398e+05,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000
50%,1.261358e+06,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.063492,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000
75%,1.903561e+06,3.000000,1.000000,0.000000,0.000000,2.000000,0.000000,1.000000,1.000000,0.000000,...,0.142857,0.103030,0.312500,0.018868,0.000000,0.000000,0.000000,0.025773,0.046875,0.000265
max,2.551470e+06,4.000000,2.000000,22.000000,4.000000,97.000000,17.000000,6.000000,4.000000,19.000000,...,0.952381,0.824242,1.000000,0.734591,0.666667,0.300000,0.809524,0.859107,0.921875,0.327815


In [373]:
# 找出numeric的feature
num_features = [ ]
for dtype, feature in zip(train.dtypes, train.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

42 Numeric Features : ['CUS_ID', 'EDUCATION_CD', 'MARRIAGE_CD', 'L1YR_A_ISSUE_CNT', 'L1YR_B_ISSUE_CNT', 'CHANNEL_A_POL_CNT', 'CHANNEL_B_POL_CNT', 'OCCUPATION_CLASS_CD', 'APC_CNT', 'INSD_CNT', 'APC_1ST_YEARDIF', 'LEVEL', 'RFM_M_LEVEL', 'ANNUAL_PREMIUM_AMT', 'AG_CNT', 'AG_NOW_CNT', 'CLC_CUR_NUM', 'ANNUAL_INCOME_AMT', 'L1YR_C_CNT', 'BANK_NUMBER_CNT', 'INSD_LAST_YEARDIF_CNT', 'BMI', 'IM_CNT', 'TERMINATION_RATE', 'TOOL_VISIT_1YEAR_CNT', 'DIEBENEFIT_AMT', 'DIEACCIDENT_AMT', 'POLICY_VALUE_AMT', 'ANNUITY_AMT', 'EXPIRATION_AMT', 'ACCIDENT_HOSPITAL_REC_AMT', 'DISEASES_HOSPITAL_REC_AMT', 'OUTPATIENT_SURGERY_AMT', 'INPATIENT_SURGERY_AMT', 'PAY_LIMIT_MED_MISC_AMT', 'FIRST_CANCER_AMT', 'ILL_ACCELERATION_AMT', 'ILL_ADDITIONAL_AMT', 'LONG_TERM_CARE_AMT', 'MONTHLY_CARE_AMT', 'LIFE_INSD_CNT', 'L1YR_GROSS_PRE_AMT']



In [374]:
# 找出連續的feature
num_continuous_features = ['APC_1ST_YEARDIF', 'ANNUAL_PREMIUM_AMT', 'ANNUAL_INCOME_AMT', 'INSD_LAST_YEARDIF_CNT',
                                     'TERMINATION_RATE', 'DIEBENEFIT_AMT', 'DIEACCIDENT_AMT', 'POLICY_VALUE_AMT', 'ANNUITY_AMT', 
                                    'EXPIRATION_AMT', 'ACCIDENT_HOSPITAL_REC_AMT', 'DISEASES_HOSPITAL_REC_AMT', 'OUTPATIENT_SURGERY_AMT', 
                                     'INPATIENT_SURGERY_AMT', 'PAY_LIMIT_MED_MISC_AMT', 'FIRST_CANCER_AMT', 'ILL_ACCELERATION_AMT', 'ILL_ADDITIONAL_AMT', 
                                     'LONG_TERM_CARE_AMT', 'MONTHLY_CARE_AMT', 'LIFE_INSD_CNT', 'L1YR_GROSS_PRE_AMT' ]
len(num_continuous_features)

22

In [375]:
# 用mice生成填好na的lists
my_imputer = IterativeImputer()
train_filled = my_imputer.fit_transform(test[num_continuous_features])

/anaconda3/lib/python3.7/site-packages/sklearn/impute/_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [376]:
# train1是continuoius numerice的dataframe
train1 = DataFrame(train_filled, columns = num_continuous_features)

In [377]:
train1

,APC_1ST_YEARDIF,ANNUAL_PREMIUM_AMT,ANNUAL_INCOME_AMT,INSD_LAST_YEARDIF_CNT,TERMINATION_RATE,DIEBENEFIT_AMT,DIEACCIDENT_AMT,POLICY_VALUE_AMT,ANNUITY_AMT,EXPIRATION_AMT,...,OUTPATIENT_SURGERY_AMT,INPATIENT_SURGERY_AMT,PAY_LIMIT_MED_MISC_AMT,FIRST_CANCER_AMT,ILL_ACCELERATION_AMT,ILL_ADDITIONAL_AMT,LONG_TERM_CARE_AMT,MONTHLY_CARE_AMT,LIFE_INSD_CNT,L1YR_GROSS_PRE_AMT
0,0.000000,0.000259,0.000423,0.000000,0.000000,0.003119,0.004404,0.001478,0.003073,-0.000096,...,0.115552,0.058259,0.162873,0.005058,0.002022,0.001988,0.004519,0.026997,0.061538,0.000151
1,0.181818,0.000245,0.000566,0.210526,0.000000,0.001354,0.002666,-0.000171,0.000985,0.000287,...,0.075337,0.047933,0.089927,0.011934,0.002244,0.001235,-0.000581,0.023352,0.015385,0.000000
2,0.068182,0.000154,0.000083,0.078947,0.000000,0.000085,0.000085,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.000090
3,0.159091,0.000260,0.000560,0.184211,0.000000,0.001228,0.002487,-0.000145,0.000932,0.000279,...,0.075775,0.046038,0.097804,0.010120,0.001790,0.001033,-0.000653,0.023603,0.015385,0.000000
4,0.045455,0.000310,0.000083,0.052632,0.000000,0.000000,0.000000,0.000000,0.000498,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.000179
5,0.022727,0.002420,0.002500,0.026316,0.000000,0.002157,0.003154,0.001423,0.002069,0.000451,...,0.086315,0.039744,0.144749,0.000648,-0.000579,0.000267,0.001443,0.023997,0.030769,0.001452
6,0.068182,0.000578,0.000833,0.078947,0.000000,0.000000,0.000000,0.000000,0.000498,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000314
7,0.136364,0.003088,0.000952,0.157895,0.000000,0.002356,0.003265,-0.001367,0.002682,0.001950,...,0.077186,0.040833,0.108357,0.006535,-0.000390,-0.000144,-0.000136,0.021911,0.015385,0.000000
8,0.000000,0.011321,0.001647,0.000000,0.000000,0.003329,0.003371,0.002688,0.003610,0.002809,...,0.055722,0.008606,0.141627,-0.010270,-0.007307,-0.003469,-0.000812,0.018103,0.000000,0.006626
9,0.159091,0.000260,0.000560,0.184211,0.000000,0.001228,0.002487,-0.000145,0.000932,0.000279,...,0.075775,0.046038,0.097804,0.010120,0.001790,0.001033,-0.000653,0.023603,0.015385,0.000000


In [378]:
train1.isna().sum()

APC_1ST_YEARDIF              0
ANNUAL_PREMIUM_AMT           0
ANNUAL_INCOME_AMT            0
INSD_LAST_YEARDIF_CNT        0
TERMINATION_RATE             0
DIEBENEFIT_AMT               0
DIEACCIDENT_AMT              0
POLICY_VALUE_AMT             0
ANNUITY_AMT                  0
EXPIRATION_AMT               0
ACCIDENT_HOSPITAL_REC_AMT    0
DISEASES_HOSPITAL_REC_AMT    0
OUTPATIENT_SURGERY_AMT       0
INPATIENT_SURGERY_AMT        0
PAY_LIMIT_MED_MISC_AMT       0
FIRST_CANCER_AMT             0
ILL_ACCELERATION_AMT         0
ILL_ADDITIONAL_AMT           0
LONG_TERM_CARE_AMT           0
MONTHLY_CARE_AMT             0
LIFE_INSD_CNT                0
L1YR_GROSS_PRE_AMT           0
dtype: int64

In [379]:
# 找出離散的features
num_discrete_features = ['EDUCATION_CD', 'MARRIAGE_CD', 'L1YR_A_ISSUE_CNT', 'L1YR_B_ISSUE_CNT', 'CHANNEL_A_POL_CNT', 
                                               'CHANNEL_B_POL_CNT', 'OCCUPATION_CLASS_CD', 'APC_CNT', 'INSD_CNT', 'LEVEL', 'RFM_M_LEVEL',
                                               'AG_CNT', 'AG_NOW_CNT', 'CLC_CUR_NUM', 'L1YR_C_CNT', 'BANK_NUMBER_CNT', 'BMI', 'IM_CNT', 
                                              'TOOL_VISIT_1YEAR_CNT']
len(num_discrete_features)

19

In [380]:
# train2是discrete numeric的dataframe
train2 = test[num_discrete_features]

In [381]:
test[num_discrete_features].isna().sum()

EDUCATION_CD            20562
MARRIAGE_CD              7951
L1YR_A_ISSUE_CNT            0
L1YR_B_ISSUE_CNT            0
CHANNEL_A_POL_CNT           0
CHANNEL_B_POL_CNT           0
OCCUPATION_CLASS_CD      3960
APC_CNT                     0
INSD_CNT                    0
LEVEL                   43305
RFM_M_LEVEL             43282
AG_CNT                      0
AG_NOW_CNT                  0
CLC_CUR_NUM                 0
L1YR_C_CNT              87936
BANK_NUMBER_CNT             0
BMI                     16645
IM_CNT                      0
TOOL_VISIT_1YEAR_CNT        0
dtype: int64

In [382]:
# 選出有缺值的discrete
num_discrete_features1 = ['EDUCATION_CD', 'MARRIAGE_CD', 'OCCUPATION_CLASS_CD', 'LEVEL', 'RFM_M_LEVEL', 
                                      'L1YR_C_CNT', 'BMI']

In [383]:
for i in range(len(num_discrete_features1)):
    print(train2[num_discrete_features1].iloc[:, i].unique())

[ 2. nan  1.  3.  4.]
[ 0. nan  1.  2.]
[ 1.  4.  2.  3. nan  5.  0.  6.]
[ 5.  1.  2.  3.  4. nan]
[ 8.  3.  5.  7.  9. 10. nan]
[nan  1.  4.  2. 11.  7. 10.  6.  9. 17.  3.  5. 12.  8. 13. 22. 14. 19.
 15. 16. 21. 47. 18. 25. 20. 24. 29. 23.]
[0.325 0.075 0.15  0.2   0.125 0.175 0.275 0.3   0.25  0.225 0.4   0.425
   nan 0.375 0.1   0.475 0.35  0.05  0.025 0.45  0.    0.575 0.525 0.5
 0.55  0.6   0.65  0.625 0.725 0.675 0.775 0.7   0.85  0.825 1.    0.75 ]


In [384]:
train2[num_discrete_features1].head(10)

,EDUCATION_CD,MARRIAGE_CD,OCCUPATION_CLASS_CD,LEVEL,RFM_M_LEVEL,L1YR_C_CNT,BMI
0,2.0,0.0,1.0,5.0,8.0,NaN,0.325
1,NaN,0.0,1.0,1.0,3.0,NaN,0.075
2,1.0,0.0,1.0,2.0,3.0,NaN,0.150
3,NaN,0.0,1.0,1.0,3.0,NaN,0.200
4,1.0,0.0,1.0,5.0,3.0,NaN,0.125
5,NaN,0.0,1.0,5.0,5.0,NaN,0.175
6,NaN,0.0,1.0,1.0,7.0,NaN,0.125
7,NaN,0.0,1.0,1.0,3.0,NaN,0.200
8,NaN,0.0,1.0,5.0,3.0,NaN,0.275
9,NaN,0.0,1.0,1.0,3.0,NaN,0.175


In [385]:
import random 
a = train2[num_discrete_features1].iloc[:, 0].unique()
cleanedList = [x for x in a if str(x) != 'nan']
random.shuffle(cleanedList)
cleanedList

[3.0, 1.0, 2.0, 4.0]

In [386]:
col_number = train2[num_discrete_features1].shape[1]
col_number

7

In [387]:
# 填值
def fillna_random_choice(x):  # x = train2[num_discrete_features1]
    col_number = x.shape[1]
 
    for col_index in range(col_number):

        #取出有NaN的index並存成list
        index = list(x[x.iloc[: , col_index].isna()].index)
        print(len(index))
        
        # 取出此column的unique值並把NaN拿掉
        a = x.iloc[:, col_index].unique()
        a1 = [y for y in a if str(y) != 'nan']
     
        
        # 算出有每個unique有幾個
        b = []
        
        for i in range(len(a1)):
            b.append(len(x[x.iloc[:, col_index] == a1[i]]))
        
    
        
        # 算出權重
        p1 = []
        
        for i in range(len(a1)):
            p1.append(b[i] / np.sum(b))
        
        
            
        # 等比例放大每個unique到list裡，再shuffle                   
        randomlist = []
                           
        for i in range(len(a1)):
            for j in range(math.ceil(len(index) * p1[i])):
                randomlist.append(a1[i])
        
        random.shuffle(randomlist)
                
        # 用fillna填值
        replacement = pd.Series(randomlist[0 : len(index)] , index = index)
        print(replacement.head())
        x.iloc[:, col_index].fillna(value = replacement , inplace = True)
    
    return x


train2 = fillna_random_choice(train2[num_discrete_features1])
train2

30295
1    3.0
3    2.0
5    4.0
6    3.0
7    3.0
dtype: float64
11470
43    0.0
52    1.0
64    1.0
66    1.0
71    1.0
dtype: float64
4925
1267    2.0
1387    1.0
1656    1.0
1698    1.0
1814    1.0
dtype: float64
64216
3252     1.0
31077    5.0
33331    5.0
33332    1.0
33333    5.0
dtype: float64
64214
33331    5.0
33332    9.0
33333    5.0
33334    9.0
33335    3.0
dtype: float64
132142
0    1.0
1    2.0
2    1.0
3    2.0
4    1.0
dtype: float64
23692
29     0.200
216    0.250
220    0.175
233    0.225
238    0.125
dtype: float64


,EDUCATION_CD,MARRIAGE_CD,OCCUPATION_CLASS_CD,LEVEL,RFM_M_LEVEL,L1YR_C_CNT,BMI
0,2.0,0.0,1.0,5.0,8.0,1.0,0.325
1,3.0,0.0,1.0,1.0,3.0,2.0,0.075
2,1.0,0.0,1.0,2.0,3.0,1.0,0.150
3,2.0,0.0,1.0,1.0,3.0,2.0,0.200
4,1.0,0.0,1.0,5.0,3.0,1.0,0.125
5,4.0,0.0,1.0,5.0,5.0,2.0,0.175
6,3.0,0.0,1.0,1.0,7.0,1.0,0.125
7,3.0,0.0,1.0,1.0,3.0,1.0,0.200
8,2.0,0.0,1.0,5.0,3.0,1.0,0.275
9,4.0,0.0,1.0,1.0,3.0,1.0,0.175


In [388]:
train2.isna().sum()

EDUCATION_CD           0
MARRIAGE_CD            0
OCCUPATION_CLASS_CD    0
LEVEL                  0
RFM_M_LEVEL            0
L1YR_C_CNT             0
BMI                    0
dtype: int64

In [389]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 7 columns):
EDUCATION_CD           150000 non-null float64
MARRIAGE_CD            150000 non-null float64
OCCUPATION_CLASS_CD    150000 non-null float64
LEVEL                  150000 non-null float64
RFM_M_LEVEL            150000 non-null float64
L1YR_C_CNT             150000 non-null float64
BMI                    150000 non-null float64
dtypes: float64(7)
memory usage: 8.0 MB


In [390]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 22 columns):
APC_1ST_YEARDIF              150000 non-null float64
ANNUAL_PREMIUM_AMT           150000 non-null float64
ANNUAL_INCOME_AMT            150000 non-null float64
INSD_LAST_YEARDIF_CNT        150000 non-null float64
TERMINATION_RATE             150000 non-null float64
DIEBENEFIT_AMT               150000 non-null float64
DIEACCIDENT_AMT              150000 non-null float64
POLICY_VALUE_AMT             150000 non-null float64
ANNUITY_AMT                  150000 non-null float64
EXPIRATION_AMT               150000 non-null float64
ACCIDENT_HOSPITAL_REC_AMT    150000 non-null float64
DISEASES_HOSPITAL_REC_AMT    150000 non-null float64
OUTPATIENT_SURGERY_AMT       150000 non-null float64
INPATIENT_SURGERY_AMT        150000 non-null float64
PAY_LIMIT_MED_MISC_AMT       150000 non-null float64
FIRST_CANCER_AMT             150000 non-null float64
ILL_ACCELERATION_AMT         150000 non-n

In [391]:
# 將兩個table合起來
train3 = pd.concat([train1, train2], axis = 1)

In [392]:
train3.head(10)

,APC_1ST_YEARDIF,ANNUAL_PREMIUM_AMT,ANNUAL_INCOME_AMT,INSD_LAST_YEARDIF_CNT,TERMINATION_RATE,DIEBENEFIT_AMT,DIEACCIDENT_AMT,POLICY_VALUE_AMT,ANNUITY_AMT,EXPIRATION_AMT,...,MONTHLY_CARE_AMT,LIFE_INSD_CNT,L1YR_GROSS_PRE_AMT,EDUCATION_CD,MARRIAGE_CD,OCCUPATION_CLASS_CD,LEVEL,RFM_M_LEVEL,L1YR_C_CNT,BMI
0,0.000000,0.000259,0.000423,0.000000,0.0,0.003119,0.004404,0.001478,0.003073,-0.000096,...,0.026997,0.061538,0.000151,2.0,0.0,1.0,5.0,8.0,1.0,0.325
1,0.181818,0.000245,0.000566,0.210526,0.0,0.001354,0.002666,-0.000171,0.000985,0.000287,...,0.023352,0.015385,0.000000,3.0,0.0,1.0,1.0,3.0,2.0,0.075
2,0.068182,0.000154,0.000083,0.078947,0.0,0.000085,0.000085,0.000000,0.000000,0.000000,...,0.000000,0.015385,0.000090,1.0,0.0,1.0,2.0,3.0,1.0,0.150
3,0.159091,0.000260,0.000560,0.184211,0.0,0.001228,0.002487,-0.000145,0.000932,0.000279,...,0.023603,0.015385,0.000000,2.0,0.0,1.0,1.0,3.0,2.0,0.200
4,0.045455,0.000310,0.000083,0.052632,0.0,0.000000,0.000000,0.000000,0.000498,0.000000,...,0.000000,0.015385,0.000179,1.0,0.0,1.0,5.0,3.0,1.0,0.125
5,0.022727,0.002420,0.002500,0.026316,0.0,0.002157,0.003154,0.001423,0.002069,0.000451,...,0.023997,0.030769,0.001452,4.0,0.0,1.0,5.0,5.0,2.0,0.175
6,0.068182,0.000578,0.000833,0.078947,0.0,0.000000,0.000000,0.000000,0.000498,0.000000,...,0.000000,0.076923,0.000314,3.0,0.0,1.0,1.0,7.0,1.0,0.125
7,0.136364,0.003088,0.000952,0.157895,0.0,0.002356,0.003265,-0.001367,0.002682,0.001950,...,0.021911,0.015385,0.000000,3.0,0.0,1.0,1.0,3.0,1.0,0.200
8,0.000000,0.011321,0.001647,0.000000,0.0,0.003329,0.003371,0.002688,0.003610,0.002809,...,0.018103,0.000000,0.006626,2.0,0.0,1.0,5.0,3.0,1.0,0.275
9,0.159091,0.000260,0.000560,0.184211,0.0,0.001228,0.002487,-0.000145,0.000932,0.000279,...,0.023603,0.015385,0.000000,4.0,0.0,1.0,1.0,3.0,1.0,0.175


In [393]:
# 將前面沒有nan的欄位加上
num_discrete_features2 = ['L1YR_A_ISSUE_CNT', 'L1YR_B_ISSUE_CNT', 'CHANNEL_A_POL_CNT', 'CHANNEL_B_POL_CNT', 'APC_CNT', 'INSD_CNT', 
                          'AG_CNT', 'AG_NOW_CNT', 'CLC_CUR_NUM', 'BANK_NUMBER_CNT', 'IM_CNT', 'TOOL_VISIT_1YEAR_CNT', 'CUS_ID']

train4 = pd.concat([train3, test[num_discrete_features2]], axis = 1)
train4.head()

,APC_1ST_YEARDIF,ANNUAL_PREMIUM_AMT,ANNUAL_INCOME_AMT,INSD_LAST_YEARDIF_CNT,TERMINATION_RATE,DIEBENEFIT_AMT,DIEACCIDENT_AMT,POLICY_VALUE_AMT,ANNUITY_AMT,EXPIRATION_AMT,...,CHANNEL_B_POL_CNT,APC_CNT,INSD_CNT,AG_CNT,AG_NOW_CNT,CLC_CUR_NUM,BANK_NUMBER_CNT,IM_CNT,TOOL_VISIT_1YEAR_CNT,CUS_ID
0,0.000000,0.000259,0.000423,0.000000,0.0,0.003119,0.004404,0.001478,0.003073,-0.000096,...,0,0,0,1,1,1,0.000,0,0,1193
1,0.181818,0.000245,0.000566,0.210526,0.0,0.001354,0.002666,-0.000171,0.000985,0.000287,...,0,0,0,1,0,0,0.125,0,0,1727
2,0.068182,0.000154,0.000083,0.078947,0.0,0.000085,0.000085,0.000000,0.000000,0.000000,...,0,0,0,2,2,1,0.125,0,0,3293
3,0.159091,0.000260,0.000560,0.184211,0.0,0.001228,0.002487,-0.000145,0.000932,0.000279,...,0,0,0,1,0,0,0.125,0,0,4328
4,0.045455,0.000310,0.000083,0.052632,0.0,0.000000,0.000000,0.000000,0.000498,0.000000,...,0,0,0,1,1,1,0.125,0,1,5555


In [395]:
train4.isna().sum()

APC_1ST_YEARDIF              0
ANNUAL_PREMIUM_AMT           0
ANNUAL_INCOME_AMT            0
INSD_LAST_YEARDIF_CNT        0
TERMINATION_RATE             0
DIEBENEFIT_AMT               0
DIEACCIDENT_AMT              0
POLICY_VALUE_AMT             0
ANNUITY_AMT                  0
EXPIRATION_AMT               0
ACCIDENT_HOSPITAL_REC_AMT    0
DISEASES_HOSPITAL_REC_AMT    0
OUTPATIENT_SURGERY_AMT       0
INPATIENT_SURGERY_AMT        0
PAY_LIMIT_MED_MISC_AMT       0
FIRST_CANCER_AMT             0
ILL_ACCELERATION_AMT         0
ILL_ADDITIONAL_AMT           0
LONG_TERM_CARE_AMT           0
MONTHLY_CARE_AMT             0
LIFE_INSD_CNT                0
L1YR_GROSS_PRE_AMT           0
EDUCATION_CD                 0
MARRIAGE_CD                  0
OCCUPATION_CLASS_CD          0
LEVEL                        0
RFM_M_LEVEL                  0
L1YR_C_CNT                   0
BMI                          0
L1YR_A_ISSUE_CNT             0
L1YR_B_ISSUE_CNT             0
CHANNEL_A_POL_CNT            0
CHANNEL_

In [394]:
import csv
export_csv = train4.to_csv (r'/Users/owner/Desktop/2019國泰大數據競賽/Cathay_BigData-Competition/Data/adjusted_numeric_test.csv', index = None, header=True)